# Mistral Fine-tuning API

Check out the docs: https://docs.mistral.ai/capabilities/finetuning/

In [1]:
!pip install mistralai pandas


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Prepare the dataset

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required jsonl format for fine-tuning.

In [3]:
import pandas as pd
df = pd.read_csv('../dataset/amazon.csv')
df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

In [4]:
!ls -lh

'ls' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


## Reformat dataset
If you upload this ultrachat_chunk_train.jsonl to Mistral API, you might encounter an error message “Invalid file format” due to data formatting issues. To reformat the data into the correct format, you can download the reformat_dataset.py script and use it to validate and reformat both the training and evaluation data:

In [5]:
# download the validation and reformat script
!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py

'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [10]:
# validate and reformat the training data
!python reformat_data.py ultrachat_chunk_train.jsonl

In [11]:
# validate the reformat the eval data
!python reformat_data.py ultrachat_chunk_eval.jsonl

In [ ]:
# df_train.iloc[3674]['messages']

IndexError: single positional indexer is out-of-bounds

## Upload dataset

In [18]:
from mistralai import Mistral
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("MISTRAL_API_KEY")

client = Mistral(api_key=api_key)

ultrachat_chunk_eval = client.files.upload(file={
    "file_name": "reformatted_ultrachat_chunk_eval.jsonl",
    "content": open("ultrachat_chunk_eval.jsonl", "rb"),
})
ultrachat_chunk_train = client.files.upload(file={
    "file_name": "reformatted_ultrachat_chunk_train.jsonl",
    "content": open("ultrachat_chunk_train.jsonl", "rb"),
})

SDKError: API error occurred: Status 422
{"detail": "Invalid file format.", "description": "Found 7 errors in this file. You can view supported formats here: https://docs.mistral.ai/capabilities/finetuning. ", "errors": [{"message": "1 validation error for FinetuningMessages messages   Input should be a valid list [type=list_type, input_value={'product_id': 'B09V2Q4QV...&s=electronics&sr=1-14'}, input_type=dict]     For further information visit https://errors.pydantic.dev/2.9/v/list_type", "line_number": 1}, {"message": "1 validation error for FinetuningMessages messages   Input should be a valid list [type=list_type, input_value={'product_id': 'B0B2DJ5RV...s=electronics&sr=1-260'}, input_type=dict]     For further information visit https://errors.pydantic.dev/2.9/v/list_type", "line_number": 2}, {"message": "1 validation error for FinetuningMessages messages   Input should be a valid list [type=list_type, input_value={'product_id': 'B00N1U9AJ...96&s=computers&sr=1-36'}, input_type=di

In [24]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [25]:
pprint(ultrachat_chunk_train)

{
    "id": "c6a8fec2-a19d-415f-ad70-c78e95325f1d",
    "object": "file",
    "bytes": 121379382,
    "created_at": 1732359131,
    "filename": "ultrachat_chunk_train.jsonl",
    "purpose": "fine-tune",
    "sample_type": "instruct",
    "source": "upload",
    "num_lines": 28156
}


In [26]:
pprint(ultrachat_chunk_eval)

{
    "id": "0d06d17b-587c-4212-81b8-82153a0e2a94",
    "object": "file",
    "bytes": 596255,
    "created_at": 1732359132,
    "filename": "ultrachat_chunk_eval.jsonl",
    "purpose": "fine-tune",
    "sample_type": "instruct",
    "source": "upload",
    "num_lines": 142
}


## Create a fine-tuning job

In [27]:
created_jobs = client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": ultrachat_chunk_train.id, "weight": 1}],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters={
    "training_steps": 10,
    "learning_rate":0.0001
    },
    auto_start=True
)
created_jobs

JobOut(id='0b0c7fe9-c539-4c83-b525-6fc25392c014', auto_start=True, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1732359149, modified_at=1732359149, training_files=['c6a8fec2-a19d-415f-ad70-c78e95325f1d'], validation_files=['0d06d17b-587c-4212-81b8-82153a0e2a94'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None))

In [28]:
pprint(created_jobs)

{
    "id": "0b0c7fe9-c539-4c83-b525-6fc25392c014",
    "auto_start": true,
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "warmup_fraction": 0.05,
        "epochs": null,
        "fim_ratio": null,
        "seq_len": 32768
    },
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1732359149,
    "modified_at": 1732359149,
    "training_files": [
        "c6a8fec2-a19d-415f-ad70-c78e95325f1d"
    ],
    "validation_files": [
        "0d06d17b-587c-4212-81b8-82153a0e2a94"
    ],
    "fine_tuned_model": null,
    "suffix": null,
    "integrations": [],
    "trained_tokens": null,
    "repositories": [],
    "metadata": {
        "expected_duration_seconds": null,
        "cost": 0.0,
        "cost_currency": null,
        "train_tokens_per_step": null,
        "train_tokens": null,
        "data_tokens": null,
        "estimated_start_time": null
    }
}


In [29]:
jobs = client.fine_tuning.jobs.list()
print(jobs)

total=5 data=[JobOut(id='0b0c7fe9-c539-4c83-b525-6fc25392c014', auto_start=True, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=0.0431941570306258, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1732359149, modified_at=1732359152, training_files=['c6a8fec2-a19d-415f-ad70-c78e95325f1d'], validation_files=['0d06d17b-587c-4212-81b8-82153a0e2a94'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=140, cost=2.5, cost_currency='EUR', train_tokens_per_step=131072, train_tokens=1310720, data_tokens=30344845, estimated_start_time=1732360978)), JobOut(id='f0c1608f-cd6d-400b-bf87-fa52ceaa1530', auto_start=False, hyperparameters=TrainingParameters(training_steps=12, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=10.0, fim_ratio=None, seq_len=327

In [30]:
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
retrieved_jobs

DetailedJobOut(id='0b0c7fe9-c539-4c83-b525-6fc25392c014', auto_start=True, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=0.0431941570306258, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1732359149, modified_at=1732359152, training_files=['c6a8fec2-a19d-415f-ad70-c78e95325f1d'], validation_files=['0d06d17b-587c-4212-81b8-82153a0e2a94'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=140, cost=2.5, cost_currency='EUR', train_tokens_per_step=131072, train_tokens=1310720, data_tokens=30344845, estimated_start_time=1732360980), events=[EventOut(name='status-updated', created_at=1732359152, data={'status': 'QUEUED'}), EventOut(name='status-updated', created_at=1732359152, data={'status': 'VALIDATED'}), EventOut(name='status-updated', created_at=1732359152,

In [32]:
import time

retrieved_job = client.fine_tuning.jobs.get(job_id = created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.fine_tuning.jobs.get(job_id = created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)



{
    "id": "0b0c7fe9-c539-4c83-b525-6fc25392c014",
    "auto_start": true,
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "warmup_fraction": 0.05,
        "epochs": 0.0431941570306258,
        "fim_ratio": null,
        "seq_len": 32768
    },
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1732359149,
    "modified_at": 1732359152,
    "training_files": [
        "c6a8fec2-a19d-415f-ad70-c78e95325f1d"
    ],
    "validation_files": [
        "0d06d17b-587c-4212-81b8-82153a0e2a94"
    ],
    "fine_tuned_model": null,
    "suffix": null,
    "integrations": [],
    "trained_tokens": null,
    "repositories": [],
    "metadata": {
        "expected_duration_seconds": 140,
        "cost": 2.5,
        "cost_currency": "EUR",
        "train_tokens_per_step": 131072,
        "train_tokens": 1310720,
        "data_tokens": 30344845,
        "estimated_start_time": 17

KeyboardInterrupt: 

In [ ]:
# List jobs
jobs = client.fine_tuning.jobs.list()
pprint(jobs)

{
    "total": 32,
    "data": [
        {
            "id": "20178c3c-d75b-428e-b20d-7d39aa2b7468",
            "auto_start": true,
            "hyperparameters": {
                "training_steps": 10,
                "learning_rate": 0.0001,
                "epochs": 0.0431941570306258,
                "fim_ratio": null
            },
            "model": "open-mistral-7b",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1721405548,
            "modified_at": 1721405693,
            "training_files": [
                "ec5af16a-77fe-4e14-ad09-47ead2848ce6"
            ],
            "validation_files": [
                "d0c643a2-a57c-4031-bda7-5c9d6c3ec3e4"
            ],
            "fine_tuned_model": "ft:open-mistral-7b:b6e34a5e:20240719:20178c3c",
            "suffix": null,
            "integrations": [],
            "trained_tokens": 1310720,
            "repositories": [],
            "metadata": {
                "expected_duration_s

In [ ]:
# Retrieve a jobs
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
pprint(retrieved_jobs)


{
    "id": "20178c3c-d75b-428e-b20d-7d39aa2b7468",
    "auto_start": true,
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001,
        "epochs": 0.0431941570306258,
        "fim_ratio": null
    },
    "model": "open-mistral-7b",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1721405548,
    "modified_at": 1721405693,
    "training_files": [
        "ec5af16a-77fe-4e14-ad09-47ead2848ce6"
    ],
    "validation_files": [
        "d0c643a2-a57c-4031-bda7-5c9d6c3ec3e4"
    ],
    "fine_tuned_model": "ft:open-mistral-7b:b6e34a5e:20240719:20178c3c",
    "suffix": null,
    "integrations": [],
    "trained_tokens": 1310720,
    "repositories": [],
    "metadata": {
        "expected_duration_seconds": 120,
        "cost": 2.6214,
        "cost_currency": "USD",
        "train_tokens_per_step": 131072,
        "train_tokens": 1310720,
        "data_tokens": 30344845,
        "estimated_start_time": null
    },
    "events": [
        {
  

## Use a fine-tuned model

In [ ]:
chat_response = client.chat.complete(
    model = retrieved_jobs.fine_tuned_model,
    messages = [{"role":'user', "content":'What is the best French cheese?'}]
)

In [ ]:
pprint(chat_response)

{
    "id": "1fac96713fd74799922712e34e009f81",
    "object": "chat.completion",
    "model": "ft:open-mistral-7b:b6e34a5e:20240719:20178c3c",
    "usage": {
        "prompt_tokens": 10,
        "completion_tokens": 73,
        "total_tokens": 83
    },
    "created": 1721405725,
    "choices": [
        {
            "index": 0,
            "finish_reason": "stop",
            "message": {
                "content": "There isn't a single \"best\" French cheese as there are hundreds of different types of cheese to choose from, each with its unique taste and texture. Some popular French cheeses include Brie, Camembert, Roquefort, Comt\u00e9, and Ch\u00e8vre. Try different cheeses to find out the one you like best!",
                "tool_calls": null,
                "prefix": false,
                "role": "assistant"
            }
        }
    ]
}


## Integration with Weights and Biases
We can also offer support for integration with Weights & Biases (W&B) to monitor and track various metrics and statistics associated with our fine-tuning jobs. To enable integration with W&B, you will need to create an account with W&B and add your W&B information in the “integrations” section in the job creation request:



In [ ]:
client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": ultrachat_chunk_train.id, "weight": 1}],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters={"training_steps": 10, "learning_rate": 0.0001},
    integrations=[
        {
            "project": "<value>",
            "api_key": "<value>",
        }
    ]
)